<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-research-and-practice/blob/main/text-similarity-works/02_aho_cora_sick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Aho-cora-sick

##Setup

In [1]:
!pip -q install pyahocorasick
!pip -q install thefuzz

     |████████████████████████████████| 106 kB 15.3 MB/s 


In [19]:
import pandas as pd
import numpy as np
import re

import pdb

import ahocorasick as ahc
from thefuzz import process
from thefuzz import fuzz

##Loading data

In [3]:
keyword_df = pd.read_csv("keywords.csv")
keyword_df.head()

,CATEGORY0,CATEGORY1,CATEGORY2,SYNID,KEYWORDS
0,Keywords,Respiratory,NaN,KW-rESPIRATORY01,Respiratory
1,Keywords,Alcohol,Alcohol,KW-ALCOHOL184,struggling with alcohol
2,Keywords,Alcohol,Alcohol,KW-ALCOHOL185,suspected alcohol abuse
3,Keywords,Alcohol,Alcohol,KW-ALCOHOL186,taking medication for alcohol
4,Keywords,Alcohol,Alcohol,KW-ALCOHOL187,tequilla


In [4]:
ocr_sample = pd.read_csv("ocr_page.csv", encoding="cp1252")
ocr_sample.head()

,PAGEID,OCR_CONTENT,LOAD_TIMESTAMP
0,104707,PACIFIC LIFE INSURANCE COMPANY\n\nLife Insuran...,26-JAN-22 07.04.57.000000000 PM
1,104708,\r\n\r\nTEMPORARY INSURANCE AGREEMENT (TIA)\r...,26-JAN-22 07.04.57.000000000 PM
2,104709,PACIFIC LIFE INSURANCE COMPANY\r\n\r\nLife Ins...,26-JAN-22 07.04.57.000000000 PM
3,104710,\n\nNON-FORFEITURE PREMIUM CESSATION OPTIONS ...,26-JAN-22 07.04.57.000000000 PM
4,104711,\n\nADDITIONAL POLICYOWNER (If more than one ...,26-JAN-22 07.04.57.000000000 PM


In [5]:
ocr_sample["OCR_CONTENT"][1]

' \r\n\r\nTEMPORARY INSURANCE AGREEMENT (TIA)\r\n\r\nQuali?cations\r\nTo qualify for temporary alcoholic hepatitis, the Proposed Insured(s) must be over 15 days of age or under age 70, and able to answer “No" to all of the\r\nquestions on the Temporary Insurance Agreement.\r\nLimits\r\n- The TIA is limited to $1,000,000 for individual products and $1,500,000 for Second-to-Die products.\r\n- For Monthly Bene?t Term Life Insurance only, the TIA is limited to the lesser ofthe Monthly Bene?t applied for or a Monthly Bene?t amount of $5,001\r\nInitial Premium\r\nThe minimum initial premium that can be submitted with the completed application and the TIA is either 1) 10% of the annual premium, or 2) greater of\r\none modal premium or three times the monthly initial minimum premium illustrated.\r\nSubmitting\r\n- It is preferred that the check for the TIA, application, and TIA form all have the same date.\r\n- If it is not possible to obtain a check at the time the application is completed, t

##Preparing query content

In [6]:
ocr_sample["OCR_CONTENT"][1].split("\n")[:10]

[' \r',
 '\r',
 'TEMPORARY INSURANCE AGREEMENT (TIA)\r',
 '\r',
 'Quali?cations\r',
 'To qualify for temporary alcoholic hepatitis, the Proposed Insured(s) must be over 15 days of age or under age 70, and able to answer “No" to all of the\r',
 'questions on the Temporary Insurance Agreement.\r',
 'Limits\r',
 '- The TIA is limited to $1,000,000 for individual products and $1,500,000 for Second-to-Die products.\r',
 '- For Monthly Bene?t Term Life Insurance only, the TIA is limited to the lesser ofthe Monthly Bene?t applied for or a Monthly Bene?t amount of $5,001\r']

In [7]:
# converting text to sentences
sentences_list = []
sentences = ocr_sample["OCR_CONTENT"][1].split("\n")
for sentence in sentences:
  if sentence != "":
    sentences_list.append(sentence)
sentences_list[:10]

[' \r',
 '\r',
 'TEMPORARY INSURANCE AGREEMENT (TIA)\r',
 '\r',
 'Quali?cations\r',
 'To qualify for temporary alcoholic hepatitis, the Proposed Insured(s) must be over 15 days of age or under age 70, and able to answer “No" to all of the\r',
 'questions on the Temporary Insurance Agreement.\r',
 'Limits\r',
 '- The TIA is limited to $1,000,000 for individual products and $1,500,000 for Second-to-Die products.\r',
 '- For Monthly Bene?t Term Life Insurance only, the TIA is limited to the lesser ofthe Monthly Bene?t applied for or a Monthly Bene?t amount of $5,001\r']

In [8]:
contents = ocr_sample["OCR_CONTENT"][1].replace("\r\n", "")
contents

' TEMPORARY INSURANCE AGREEMENT (TIA)Quali?cationsTo qualify for temporary alcoholic hepatitis, the Proposed Insured(s) must be over 15 days of age or under age 70, and able to answer “No" to all of thequestions on the Temporary Insurance Agreement.Limits- The TIA is limited to $1,000,000 for individual products and $1,500,000 for Second-to-Die products.- For Monthly Bene?t Term Life Insurance only, the TIA is limited to the lesser ofthe Monthly Bene?t applied for or a Monthly Bene?t amount of $5,001Initial PremiumThe minimum initial premium that can be submitted with the completed application and the TIA is either 1) 10% of the annual premium, or 2) greater ofone modal premium or three times the monthly initial minimum premium illustrated.Submitting- It is preferred that the check for the TIA, application, and TIA form all have the same date.- If it is not possible to obtain a check at the time the application is completed, the check may be dated up to 3 days later than the TIA and ap

##Preparing keyword dictionary

In [9]:
def make_keyword_dictionary(df):
  keywords = []
  for _ in range(50):
    for _, row in df.iterrows():
      keywords.append((row["KEYWORDS"], (row["CATEGORY0"], row["CATEGORY1"], row["CATEGORY2"])))
  return keywords

In [10]:
def make_keyword_dictionary2(df):
  keywords = []
  for _ in range(50):
    for _, row in df.iterrows():
      keywords.append((f" {row['KEYWORDS']} ", (row["CATEGORY0"], row["CATEGORY1"], row["CATEGORY2"])))
  return keywords

In [11]:
keywords = make_keyword_dictionary(keyword_df)

In [12]:
len(keywords)

14250

In [13]:
def make_aho_automaton(keywords):
  # initialize
  A = ahc.Automaton()
  for key, cat in keywords:
    # add keys and categories
    A.add_word(key, (cat, key))
  # generate automaton
  A.make_automaton()
  return A

In [14]:
A = make_aho_automaton(keywords)

##Search keyword

In [15]:
%time
contents = "struggling with alcohol in India"
for item in A.iter(contents):
  # pdb.set_trace();
  print(item, contents[:item[0]+1])

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs
(22, (('Keywords', 'Alcohol', 'Alcohol'), 'struggling with alcohol')) struggling with alcohol


In [16]:
print("Category \t\t\t\t\t\t Keyword")
for content in sentences_list:
  for item in A.iter(contents):
    print(f"{item} >> {contents[:item[0]+1]}")
    #print(item, contents[:item[0]+1])

Category 						 Keyword
(22, (('Keywords', 'Alcohol', 'Alcohol'), 'struggling with alcohol')) >> struggling with alcohol
(22, (('Keywords', 'Alcohol', 'Alcohol'), 'struggling with alcohol')) >> struggling with alcohol
(22, (('Keywords', 'Alcohol', 'Alcohol'), 'struggling with alcohol')) >> struggling with alcohol
(22, (('Keywords', 'Alcohol', 'Alcohol'), 'struggling with alcohol')) >> struggling with alcohol
(22, (('Keywords', 'Alcohol', 'Alcohol'), 'struggling with alcohol')) >> struggling with alcohol
(22, (('Keywords', 'Alcohol', 'Alcohol'), 'struggling with alcohol')) >> struggling with alcohol
(22, (('Keywords', 'Alcohol', 'Alcohol'), 'struggling with alcohol')) >> struggling with alcohol
(22, (('Keywords', 'Alcohol', 'Alcohol'), 'struggling with alcohol')) >> struggling with alcohol
(22, (('Keywords', 'Alcohol', 'Alcohol'), 'struggling with alcohol')) >> struggling with alcohol
(22, (('Keywords', 'Alcohol', 'Alcohol'), 'struggling with alcohol')) >> struggling with alcohol
(22, (

In [17]:
# let’s  finds keywords in text
def find_keywords(line, A):
  found_keywords = []
  for end_index, (cat, keyw) in A.iter(line):
    found_keywords.append(keyw)
  return found_keywords

In [18]:
# Searching for keywords yields a list of the found keywords. 
for line in sentences_list:
  found_keywords = []
  for end_index, (cat, keyw) in A.iter(line):
    #pdb.set_trace();
    found_keywords.append(keyw)

In [19]:
# Searching for keywords yields a list of the found keywords. 
for line in sentences_list:
  key_found = find_keywords(line, A)
  if key_found != []:
    print(f"{line} : {key_found}" )

 : ['alcoholic', 'alcoholic hepatitis']


In [20]:
A=ahc.Automaton()
A.add_word("Europe",        ('VOCAB','Europe'))
A.add_word("European Union",('VOCAB','European Union'))
A.add_word("Boris Johnson", ('PERSON','Boris Johnson'))
A.add_word("Boris",         ('PERSON','Boris Johnson'))
A.add_word("boris johnson", ('PERSON','Boris Johnson (LC)'))
 
A.make_automaton()

q='Boris Johnson went off to Europe to complain about the European Union'
for item in A.iter(q):
  print(item, q[:item[0]+1])

(4, ('PERSON', 'Boris Johnson')) Boris
(12, ('PERSON', 'Boris Johnson')) Boris Johnson
(31, ('VOCAB', 'Europe')) Boris Johnson went off to Europe
(60, ('VOCAB', 'Europe')) Boris Johnson went off to Europe to complain about the Europe
(68, ('VOCAB', 'European Union')) Boris Johnson went off to Europe to complain about the European Union


In [21]:
for item in A.iter(q):
  # print(item)
  print(q[:item[0]])

Bori
Boris Johnso
Boris Johnson went off to Europ
Boris Johnson went off to Europe to complain about the Europ
Boris Johnson went off to Europe to complain about the European Unio


##fuzzywuzzy

https://github.com/seatgeek/thefuzz

In [22]:
keyword_df["KEYWORDS"].head()

0                      Respiratory
1          struggling with alcohol
2          suspected alcohol abuse
3    taking medication for alcohol
4                         tequilla
Name: KEYWORDS, dtype: object

In [23]:
def make_keyword_list(df):
  keywords = []
  for _, row in df.iterrows():
    keywords.append(row["KEYWORDS"])
  return keywords

keywords = make_keyword_list(keyword_df)
keywords[:10]

['Respiratory',
 'struggling with alcohol',
 'suspected alcohol abuse',
 'taking medication for alcohol',
 'tequilla',
 'tipsy',
 'toxic effect of unspecified alcohol',
 'under the influence',
 'under the influence of alcohol',
 'unspecified alcohol disorder']

In [52]:
process.extract(contents, keywords, limit=10)

[('struggling with alcohol', 95),
 ('abstains from alcohol', 86),
 ('abusing alcohol', 86),
 ('acting psychotic while under the influence of alcohol', 86),
 ('acting psychotic while withdrawing from alcohol dependence', 86),
 ('alcohol abuse', 86),
 ('alcohol consumption', 86),
 ('alcohol counselling', 86),
 ('alcohol craving', 86),
 ('alcohol dependence', 86)]

In [51]:
process.extract(contents, keywords, limit=10)

[('struggling with alcohol', 95),
 ('abstains from alcohol', 86),
 ('abusing alcohol', 86),
 ('acting psychotic while under the influence of alcohol', 86),
 ('acting psychotic while withdrawing from alcohol dependence', 86),
 ('alcohol abuse', 86),
 ('alcohol consumption', 86),
 ('alcohol counselling', 86),
 ('alcohol craving', 86),
 ('alcohol dependence', 86)]

In [26]:
process.extractOne(contents, keywords)

('struggling with alcohol', 95)

In [27]:
process.extractBests(contents, keywords)

[('struggling with alcohol', 95),
 ('abstains from alcohol', 86),
 ('abusing alcohol', 86),
 ('acting psychotic while under the influence of alcohol', 86),
 ('acting psychotic while withdrawing from alcohol dependence', 86)]

##Sample search

In [8]:
sample_keywords = [
  "gallbladder", "gallbladder wall", "cholelithiasis", "pericholecystic fluid", "unremarkable", 
  "without evidence","normal in thickness", "unremarkable", "without evidence", "normal in thickness"
]

In [67]:
ocr_text = """
The gallbladder is unremarkable without evidence of cholelithiasis or pericholecystic fluid. The gallbladder wall is normal in thickness.
"""

process.extractBests(ocr_text, sample_keywords, limit=100)

[('pericholecystic fluid', 90),
 ('normal in thickness', 90),
 ('normal in thickness', 90),
 ('gallbladder', 60),
 ('gallbladder wall', 60),
 ('cholelithiasis', 60),
 ('unremarkable', 60),
 ('without evidence', 60),
 ('unremarkable', 60),
 ('without evidence', 60)]

In [42]:
ocr_text2 = """
The gallbladder is unremarkable without evidence of cholelithiasis or pericholecystic fluid. The gallbladder wall is normal in thickness.
"""

print(ocr_text2)
process.extract(ocr_text2, sample_keywords, scorer=fuzz.token_sort_ratio, limit=10)


The gallbladder is unremarkable without evidence of cholelithiasis or pericholecystic fluid. The gallbladder wall is normal in thickness.



[('pericholecystic fluid', 27),
 ('normal in thickness', 25),
 ('normal in thickness', 25),
 ('gallbladder wall', 21),
 ('without evidence', 21),
 ('without evidence', 21),
 ('cholelithiasis', 19),
 ('unremarkable', 16),
 ('unremarkable', 16),
 ('gallbladder', 15)]

In [24]:
terms=['Houses of Parliament', 'Diane Abbott', 'Boris Johnson']
 
q= "Diane Jafer, Theresa May and Boris Johnstone were in the Houses of Parliament the other day"
process.extract(q,terms, scorer=fuzz.WRatio)

[('Houses of Parliament', 90), ('Diane Abbott', 86), ('Boris Johnson', 86)]

In [54]:
process.extract(ocr_text, sample_keywords)

[('cholelithiasis', 90),
 ('pericholecystic fluid', 90),
 ('unremarkable', 90),
 ('without evidence', 90),
 ('unremarkable', 90)]

In [57]:
set(process.extract(ocr_text, sample_keywords, limit=10))

{('cholelithiasis', 90),
 ('gallbladder', 60),
 ('gallbladder wall', 68),
 ('normal in thickness', 42),
 ('pericholecystic fluid', 90),
 ('unremarkable', 90),
 ('without evidence', 90)}

In [33]:
sample_keyword_groups ={
    "site_location": ["gallbladder", "gallbladder wall"],
    "disease_injury_condition": ["cholelithiasis", "pericholecystic fluid"],
    "findings_observations": ["unremarkable", "without evidence","normal in thickness"],
    "codes": ["ICD10", "LOINC", "RXNORM", "CPT"]
}

In [39]:
for item in sample_keyword_groups.items():
  print(set(process.extractBests(ocr_text, item[1])))

{('gallbladder wall', 60), ('gallbladder', 60)}
{('cholelithiasis', 60), ('pericholecystic fluid', 90)}
{('unremarkable', 60), ('normal in thickness', 90), ('without evidence', 60)}
{('RXNORM', 40), ('LOINC', 36), ('CPT', 20), ('ICD10', 24)}
